# Übungsaufgaben 8

## Aufgabe 2: xmltcf (Verarbeitung von annotierten XML-Dateien im TCF-Format)

Im deutschen Textarchiv findet man auch annotierte Daten zu den
entsprechenden Dokumenten.  Laden Sie eine solche lemmatisierte und
annotierte XML-Datei im [TCF-Format](https://weblicht.sfs.uni-tuebingen.de/weblichtwiki/index.php/The_TCF_Format) (Text Corpus Format)
herunter (z.B. Altmann: https://www.deutschestextarchiv.de/book/download_fulltcf/16299)
und schreiben Sie ein Programm `xmltcf`, das eine zeilenweise die Sätze
ausgibt, wobei die Token entweder mit ihren Lemmata oder ihren Tags
annotiert sind:
```bash
$ ./xmltcf.py file.xml
Ich/ich gehe/gehen ...
$ ./xmltcf.py --tags file.xml
Der/DET Mann/NOUN ...
```


### Vorgehen:

- Iterieren über Sätze (`<sentences>`) und Tokens (`<tokens>`) des TCF-XML-Files
- Lookup von POS (`<POStags>`) und Lemmata (`<lemmas>`) im XML über `tokenIDs`

#### TCF-XML-Grundstruktur:
```xml
<D-Spin xmlns="http://www.dspin.de/data" version="0.4">
    <MetaData xmlns="http://www.dspin.de/data/metadata">
        ...
    </MetaData>
    <TextCorpus  xmlns="http://www.dspin.de/data/textcorpus" lang="de">
        <tokens>
            <token ID="w1">zwei</token>
            <token ID="w2">Tests</token>
        </tokens>
        <sentences>
            <sentence ID="s1" tokenIDs="w1 w2"/>
        </sentences>
        <lemmas>
            <lemma tokenIDs="w1">zwei</lemma>
            <lemma tokenIDs="w2">Test</lemma>
        </lemmas>
        <POStags tagset="stts">
            <tag tokenIDs="w1">ART</tag>
            <tag tokenIDs="w2">NN</tag>
        </POStags>
        <orthography>
            ...
        </orthography>
    </TextCorpus>
</D-Spin>
```

#### xmltcf.py:

In [ ]:
#!/usr/bin/env python3

import urllib.request
import xml.etree.ElementTree as ET
import argparse

# Token-Daten-Klasse: speichert Token, ID, POS-Tag und Lemma
class Token:
    def __init__(self, word, id, tag=None, lemma=None):
        self.word = word
        self.id = id
        self.tag = tag
        self.lemma = lemma

# url = 'https://www.deutschestextarchiv.de/book/download_fulltcf/16299'

def parse_xml(url, tags):
    with urllib.request.urlopen(url) as f:
        root = ET.fromstring(f.read())

#    with open(name, encoding='utf-8') as f:
#        root = ET.fromstring(f.read())
        
    #Namespace-Dictionary (TextCorpus-Namespace):
    ns = {'tc': "http://www.dspin.de/data/textcorpus"} 
    
    #1. Iterieren über Token-Elemente im TextCorpus-Knoten, Generierung lex-Dictionary mit Tokens+IDs:
    # (Verwendung von Token-Datenklasse (oben definiert) in Dictionary-Comprehension)
    lex = {t.attrib["ID"]: Token(t.text, t.attrib["ID"]) for t in root.find('tc:TextCorpus', ns).find('tc:tokens', ns)}

    #2. Iterieren über Lemmas- und POSTags-Elemente und Ergänzung in lex-Dictionary (über tokenIDs):
    for lemma in root.find('tc:TextCorpus', ns).find('tc:lemmas', ns ):
        token = lex[lemma.attrib["tokenIDs"]]
        token.lemma = lemma.text
    for tag in root.find('tc:TextCorpus', ns ).find('tc:POStags', ns):
        token = lex[tag.attrib["tokenIDs"]]
        token.tag = tag.text
        
    # lex-Dictionary enthält für jede Token-ID: word, id, pos-tag, lemma

    #3. Iterieren über sentences-Knoten und satzweises Erzeugen von "Token/Tag"- bzw. "Token/Lemma"-Strings: 
    # (Lookup in lex-Dictionary über tokenIDs)
    #(in Abhängigkeit von argparse-Option -t = tags bei Ausführung):
    for sent in root.find('tc:TextCorpus', ns).find('tc:sentences', ns):
        if tags:
            print(" ".join([lex[id].word + "/" + lex[id].tag for id in sent.attrib["tokenIDs"].split(" ")]))
        else:
            print(" ".join([lex[id].word + "/" + lex[id].lemma for id in sent.attrib["tokenIDs"].split(" ")]))


# Argumente für Programmaufruf über Kommandozeile (inklusive Help-Option):
parser = argparse.ArgumentParser(description='write tokens with their tags or lemmas from tcf files')
parser.add_argument('-t', '--tags', help='output tokens with their tag instead of their lemma', action="store_true")
parser.add_argument('urls', help='list of urls to operate on', nargs='*')

args = parser.parse_args()
for url in args.urls:
    parse_xml(url, args.tags)


#### Usage:

In [9]:
%%bash
./xmltcf.py -h

usage: xmltcf.py [-h] [-t] [urls [urls ...]]

write tokens with their tags or lemmas from tcf files

positional arguments:
  urls        list of urls to operate on

optional arguments:
  -h, --help  show this help message and exit
  -t, --tags  output tokens with their tag instead of their lemma


#### Ausgabe Sätze mit POS-Tags:

In [22]:
%%bash
./xmltcf.py -t https://www.deutschestextarchiv.de/book/download_fulltcf/16299 > sents.txt

In [25]:
%%bash
sed 10q sents.txt

DIE/ART ELEMENTARORGANISMEN/NN UND/KON IHRE/ADJA BEZIEHUNGEN/NN ZU/APPR DEN/ART ZELLEN/NN ./$. VON/NN RICHARD/NE ALTMANN/NE ./$.
MIT/NE ZWEI/ADJA ABBILDUNGEN/NN IM/APPR TEXT/NE UND/KON XXI/NE TAFELN./NE LEIPZIG/NE ,/$, VERLAG/NN VON/NE VEIT/NE &/$( COMP./NE 1890/CARD ./$.
DIE/ART ELEMENTARORGANISMEN/NN UND/KON IHRE/ADJA BEZIEHUNGEN/NN ZU/APPR DEN/ART ZELLEN/NN ./$. VON/NN RICHARD/NE ALTMANN/NE ./$.
MIT/NE ZWEI/ADJA ABBILDUNGEN/NN IM/APPR TEXT/NE UND/KON XXI/NE TAFELN./NE LEIPZIG/NE ,/$, VERLAG/NN VON/NE VEIT/NE &/$( COMP./NE 1890/CARD ./$.
HERRN/NN WILHELM/NE HIS/NE IN/APPR DANKBARER/ADJA VEREHRUNG/NN GEWIDMET/VVPP VOM/ADJA VERFASSER/NN ./$.
Vorbemerkung/NN ./$.
Die/ART nachfolgenden/ADJA Capitel/NN enthalten/VVFIN im/APPRART Wesentlichen/NN eine/ART theils/ADV erweiterte/ADJA ,/$, theils/ADV verkürzte/ADJA Zusammenstellung/NN derjenigen/PDS Abhandlungen/NN ,/$, welche/PRELS bisher/ADV von/APPR mir/PPER über/APPR die/ART Zellengranula/NN veröffentlicht/VVPP worden/VAPP sind/VAFIN ./$.


#### Ausgabe Sätze mit Lemmata:

In [26]:
%%bash
./xmltcf.py https://www.deutschestextarchiv.de/book/download_fulltcf/16299 > sents_lemmata.txt

In [27]:
%%bash
sed 10q sents_lemmata.txt

DIE/d ELEMENTARORGANISMEN/Elementarorganismus UND/und IHRE/Ihre BEZIEHUNGEN/Beziehung ZU/zu DEN/d ZELLEN/Zelle ./. VON/VON RICHARD/Richard ALTMANN/Altmann ./.
MIT/Massachussetts_Institute_Of_Technology ZWEI/zwei ABBILDUNGEN/Abbildung IM/IM TEXT/Text UND/und XXI/Xxi TAFELN./TAFELN. LEIPZIG/Leipzig ,/, VERLAG/Verlag VON/VON VEIT/Veit &/& COMP./COMP. 1890/1890 ./.
DIE/d ELEMENTARORGANISMEN/Elementarorganismus UND/und IHRE/Ihre BEZIEHUNGEN/Beziehung ZU/zu DEN/d ZELLEN/Zelle ./. VON/VON RICHARD/Richard ALTMANN/Altmann ./.
MIT/Massachussetts_Institute_Of_Technology ZWEI/zwei ABBILDUNGEN/Abbildung IM/IM TEXT/Text UND/und XXI/Xxi TAFELN./TAFELN. LEIPZIG/Leipzig ,/, VERLAG/Verlag VON/VON VEIT/Veit &/& COMP./COMP. 1890/1890 ./.
HERRN/Herr WILHELM/Wilhelm HIS/His IN/in DANKBARER/dankbar VEREHRUNG/Verehrung GEWIDMET/widmen VOM/vom VERFASSER/Verfasser ./.
Vorbemerkung/Vorbemerkung ./.
Die/d nachfolgenden/nachfolgend Capitel/Kapitel enthalten/enthalten im/im Wesentlichen/Wesentliche eine/eine theils